In [ ]:
import chromadb
#https://github.com/chroma-core/chroma/blob/main/chromadb/experimental/density_relevance.ipynb

client =  chromadb.PersistentClient('./chroma_docs')
col_raw = client.get_collection('documents_raw')
col_summary = client.get_collection('documents_summary')
data_raw = col_raw.get()
data = col_summary.get()

print(data.keys())
documents = data['documents']
documents_raw = data_raw['documents']
len(data['documents'])


In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd

embedding = SentenceTransformer("all-MiniLM-L6-v2")


In [ ]:
from top2vec import Top2Vec

t2v = Top2Vec(documents=documents,embedding_model=embedding.encode)

In [ ]:
t2v_topics , score , id = t2v.get_topics()

print(list(zip(t2v_topics,score)))


In [ ]:
for i in range(5):
    print(t2v_topics[t2v.doc_top[i]][:5])
    print(documents[i])
    pass

for t in t2v_topics:
    t2v_labels= '_'.join(t[:5])
    print(t2v_labels)

In [ ]:
from bertopic import BERTopic
bt = BERTopic()
documents_embedding = embedding.encode(documents)
topics, probs = bt.fit_transform(documents=data['documents'],embeddings=documents_embedding)
bt.get_topic_info()
bt.visualize_topics()
hierarchical_topics = bt.hierarchical_topics(data['documents'])
bt.visualize_hierarchy(hierarchical_topics=hierarchical_topics)


In [ ]:
bt.visualize_document_datamap(documents,topics,documents_embedding)


In [ ]:
bt.visualize_topics()

In [ ]:
import datamapplot
from umap import UMAP

umap_model = UMAP(n_neighbors=15, n_components=2, min_dist=0.15, metric='cosine').fit(documents_embedding)
embeddings_2d = umap_model.embedding_


"""width = 1200
height = 1200
figure, axes = datamapplot.create_plot(
        embeddings_2d,
        num
        figsize=(width/100, height/100),
        dpi=100,
        title="title",
        sub_title="sub_title",
    )"""


In [ ]:
from utils import sys_msg,completion

sys_msg = "You are an expert ontology and entity designer."

prompt ="""Exctract entities, types, relationships for the following document.
{text}
"""
text = ""
for i,d in enumerate(documents[:5]):
    text += f"""### {i}:\n{d.strip()}\n"""
    break

prompt = prompt.format(text=text)
print(prompt)

In [ ]:
completion(prompt)

In [ ]:
from graphrag.index.graph.extractors.graph.graph_extractor import GraphExtractor

e = GraphExtractor(completion,)


In [ ]:

#One of the following types: [{entity_types}]
entity_prompt = """
-Goal-
Given a text document that is potentially relevant to this activity and a list of entity types, identify all entities of those types from the text and all relationships among the identified entities.
 
-Steps-
1. Identify all entities. For each identified entity, extract the following information:
- entity_name: Name of the entity, capitalized
- entity_type: Type of the entity
- entity_description: Comprehensive description of the entity's attributes and activities
Format each entity as ("entity"{tuple_delimiter}<entity_name>{tuple_delimiter}<entity_type>{tuple_delimiter}<entity_description>)
 
2. From the entities identified in step 1, identify all pairs of (source_entity, target_entity) that are *clearly related* to each other.
For each pair of related entities, extract the following information:
- source_entity: name of the source entity, as identified in step 1
- target_entity: name of the target entity, as identified in step 1
- relationship_description: explanation as to why you think the source entity and the target entity are related to each other
- relationship_strength: a numeric score indicating strength of the relationship between the source entity and target entity
 Format each relationship as ("relationship"{tuple_delimiter}<source_entity>{tuple_delimiter}<target_entity>{tuple_delimiter}<relationship_description>{tuple_delimiter}<relationship_strength>)
 
3. Return output in English as a single list of all the entities and relationships identified in steps 1 and 2. Use **{record_delimiter}** as the list delimiter.
 
4. When finished, output {completion_delimiter}

-Real Data-
######################
Entity_types: {entity_types}
Text: {input_text}
######################
Output:"""

EXAMPLE = """######################
-Examples-
######################
Example 1:
Entity_types: ORGANIZATION,PERSON
Text:
The Verdantis's Central Institution is scheduled to meet on Monday and Thursday, with the institution planning to release its latest policy decision on Thursday at 1:30 p.m. PDT, followed by a press conference where Central Institution Chair Martin Smith will take questions. Investors expect the Market Strategy Committee to hold its benchmark interest rate steady in a range of 3.5%-3.75%.
######################
Output:
("entity"{tuple_delimiter}CENTRAL INSTITUTION{tuple_delimiter}ORGANIZATION{tuple_delimiter}The Central Institution is the Federal Reserve of Verdantis, which is setting interest rates on Monday and Thursday)
{record_delimiter}
("entity"{tuple_delimiter}MARTIN SMITH{tuple_delimiter}PERSON{tuple_delimiter}Martin Smith is the chair of the Central Institution)
{record_delimiter}
("entity"{tuple_delimiter}MARKET STRATEGY COMMITTEE{tuple_delimiter}ORGANIZATION{tuple_delimiter}The Central Institution committee makes key decisions about interest rates and the growth of Verdantis's money supply)
{record_delimiter}
("relationship"{tuple_delimiter}MARTIN SMITH{tuple_delimiter}CENTRAL INSTITUTION{tuple_delimiter}Martin Smith is the Chair of the Central Institution and will answer questions at a press conference{tuple_delimiter}9)
{completion_delimiter}

######################
Example 2:
Entity_types: ORGANIZATION
Text:
TechGlobal's (TG) stock skyrocketed in its opening day on the Global Exchange Thursday. But IPO experts warn that the semiconductor corporation's debut on the public markets isn't indicative of how other newly listed companies may perform.

TechGlobal, a formerly public company, was taken private by Vision Holdings in 2014. The well-established chip designer says it powers 85% of premium smartphones.
######################
Output:
("entity"{tuple_delimiter}TECHGLOBAL{tuple_delimiter}ORGANIZATION{tuple_delimiter}TechGlobal is a stock now listed on the Global Exchange which powers 85% of premium smartphones)
{record_delimiter}
("entity"{tuple_delimiter}VISION HOLDINGS{tuple_delimiter}ORGANIZATION{tuple_delimiter}Vision Holdings is a firm that previously owned TechGlobal)
{record_delimiter}
("relationship"{tuple_delimiter}TECHGLOBAL{tuple_delimiter}VISION HOLDINGS{tuple_delimiter}Vision Holdings formerly owned TechGlobal from 2014 until present{tuple_delimiter}5)
{completion_delimiter}

######################
Example 3:
Entity_types: ORGANIZATION,GEO,PERSON
Text:
Five Aurelians jailed for 8 years in Firuzabad and widely regarded as hostages are on their way home to Aurelia.

The swap orchestrated by Quintara was finalized when $8bn of Firuzi funds were transferred to financial institutions in Krohaara, the capital of Quintara.

The exchange initiated in Firuzabad's capital, Tiruzia, led to the four men and one woman, who are also Firuzi nationals, boarding a chartered flight to Krohaara.

They were welcomed by senior Aurelian officials and are now on their way to Aurelia's capital, Cashion.

The Aurelians include 39-year-old businessman Samuel Namara, who has been held in Tiruzia's Alhamia Prison, as well as journalist Durke Bataglani, 59, and environmentalist Meggie Tazbah, 53, who also holds Bratinas nationality.
######################
Output:
("entity"{tuple_delimiter}FIRUZABAD{tuple_delimiter}GEO{tuple_delimiter}Firuzabad held Aurelians as hostages)
{record_delimiter}
("entity"{tuple_delimiter}AURELIA{tuple_delimiter}GEO{tuple_delimiter}Country seeking to release hostages)
{record_delimiter}
("entity"{tuple_delimiter}QUINTARA{tuple_delimiter}GEO{tuple_delimiter}Country that negotiated a swap of money in exchange for hostages)
{record_delimiter}
{record_delimiter}
("entity"{tuple_delimiter}TIRUZIA{tuple_delimiter}GEO{tuple_delimiter}Capital of Firuzabad where the Aurelians were being held)
{record_delimiter}
("entity"{tuple_delimiter}KROHAARA{tuple_delimiter}GEO{tuple_delimiter}Capital city in Quintara)
{record_delimiter}
("entity"{tuple_delimiter}CASHION{tuple_delimiter}GEO{tuple_delimiter}Capital city in Aurelia)
{record_delimiter}
("entity"{tuple_delimiter}SAMUEL NAMARA{tuple_delimiter}PERSON{tuple_delimiter}Aurelian who spent time in Tiruzia's Alhamia Prison)
{record_delimiter}
("entity"{tuple_delimiter}ALHAMIA PRISON{tuple_delimiter}GEO{tuple_delimiter}Prison in Tiruzia)
{record_delimiter}
("entity"{tuple_delimiter}DURKE BATAGLANI{tuple_delimiter}PERSON{tuple_delimiter}Aurelian journalist who was held hostage)
{record_delimiter}
("entity"{tuple_delimiter}MEGGIE TAZBAH{tuple_delimiter}PERSON{tuple_delimiter}Bratinas national and environmentalist who was held hostage)
{record_delimiter}
("relationship"{tuple_delimiter}FIRUZABAD{tuple_delimiter}AURELIA{tuple_delimiter}Firuzabad negotiated a hostage exchange with Aurelia{tuple_delimiter}2)
{record_delimiter}
("relationship"{tuple_delimiter}QUINTARA{tuple_delimiter}AURELIA{tuple_delimiter}Quintara brokered the hostage exchange between Firuzabad and Aurelia{tuple_delimiter}2)
{record_delimiter}
("relationship"{tuple_delimiter}QUINTARA{tuple_delimiter}FIRUZABAD{tuple_delimiter}Quintara brokered the hostage exchange between Firuzabad and Aurelia{tuple_delimiter}2)
{record_delimiter}
("relationship"{tuple_delimiter}SAMUEL NAMARA{tuple_delimiter}ALHAMIA PRISON{tuple_delimiter}Samuel Namara was a prisoner at Alhamia prison{tuple_delimiter}8)
{record_delimiter}
("relationship"{tuple_delimiter}SAMUEL NAMARA{tuple_delimiter}MEGGIE TAZBAH{tuple_delimiter}Samuel Namara and Meggie Tazbah were exchanged in the same hostage release{tuple_delimiter}2)
{record_delimiter}
("relationship"{tuple_delimiter}SAMUEL NAMARA{tuple_delimiter}DURKE BATAGLANI{tuple_delimiter}Samuel Namara and Durke Bataglani were exchanged in the same hostage release{tuple_delimiter}2)
{record_delimiter}
("relationship"{tuple_delimiter}MEGGIE TAZBAH{tuple_delimiter}DURKE BATAGLANI{tuple_delimiter}Meggie Tazbah and Durke Bataglani were exchanged in the same hostage release{tuple_delimiter}2)
{record_delimiter}
("relationship"{tuple_delimiter}SAMUEL NAMARA{tuple_delimiter}FIRUZABAD{tuple_delimiter}Samuel Namara was a hostage in Firuzabad{tuple_delimiter}2)
{record_delimiter}
("relationship"{tuple_delimiter}MEGGIE TAZBAH{tuple_delimiter}FIRUZABAD{tuple_delimiter}Meggie Tazbah was a hostage in Firuzabad{tuple_delimiter}2)
{record_delimiter}
("relationship"{tuple_delimiter}DURKE BATAGLANI{tuple_delimiter}FIRUZABAD{tuple_delimiter}Durke Bataglani was a hostage in Firuzabad{tuple_delimiter}2)
{completion_delimiter}

###################### 
"""
DEFAULT_TUPLE_DELIMITER = "<|>"
DEFAULT_RECORD_DELIMITER = "##"
DEFAULT_COMPLETION_DELIMITER = "<|COMPLETE|>"
DEFAULT_ENTITY_TYPES = "everything" #["models","ways","casualty"]#["organization", "person", "geo", "event"]

entity_prompt = entity_prompt.format(record_delimiter=DEFAULT_RECORD_DELIMITER,
                     tuple_delimiter=DEFAULT_TUPLE_DELIMITER,
                     entity_types=DEFAULT_ENTITY_TYPES,
                     completion_delimiter = DEFAULT_COMPLETION_DELIMITER,
                     input_text=documents[0])

print(entity_prompt)


In [16]:
entity_prompt = """
-Goal-
Given a text document that is potentially relevant to this activity and a list of entity types, identify all entities of those types from the text and all relationships among the identified entities.
 
-Steps-
1. Identify all entities. For each identified entity, extract the following information in JSON format:
entities:
- entity_name: Name of the entity, capitalized
- entity_type: Type of the entity
- entity_description: Comprehensive description of the entity's attributes and activities
 
2. From the entities identified in step 1, identify all pairs of (source_entity, target_entity) that are *clearly related* to each other.
For each pair of related entities, extract the following information:
relationships:
- source_entity: name of the source entity, as identified in step 1
- target_entity: name of the target entity, as identified in step 1
- relationship_description: explanation as to why you think the source entity and the target entity are related to each other
- relationship_strength: a numeric score indicating strength of the relationship between the source entity and target entity
 
3. Return output in English as a single list of all the entities and relationships identified in steps 1 and 2.
 
######################
Entity_types: {entity_types}
Text: {input_text}
######################
Output:"""

entity_prompt = entity_prompt.format(entity_types="Things",input_text=documents[0])
print(entity_prompt)



-Goal-
Given a text document that is potentially relevant to this activity and a list of entity types, identify all entities of those types from the text and all relationships among the identified entities.
 
-Steps-
1. Identify all entities. For each identified entity, extract the following information in JSON format:
entities:
- entity_name: Name of the entity, capitalized
- entity_type: Type of the entity
- entity_description: Comprehensive description of the entity's attributes and activities
 
2. From the entities identified in step 1, identify all pairs of (source_entity, target_entity) that are *clearly related* to each other.
For each pair of related entities, extract the following information:
relationships:
- source_entity: name of the source entity, as identified in step 1
- target_entity: name of the target entity, as identified in step 1
- relationship_description: explanation as to why you think the source entity and the target entity are related to each other
- relation

In [18]:
results = completion(entity_prompt)
results

{'entities': [{'entity_name': 'WizardLM',
   'entity_type': 'Things',
   'entity_description': 'A new large language model (LLM) trained on a dataset of instructions generated by Evol-Instruct, which outperforms ChatGPT in some tasks.'},
  {'entity_name': 'Evol-Instruct',
   'entity_type': 'Things',
   'entity_description': 'An AI system that creates complex instructions by gradually rewriting simpler ones. The instructions generated by Evol-Instruct are better than those created by humans.'},
  {'entity_name': 'ChatGPT',
   'entity_type': 'Things',
   'entity_description': 'A large language model developed by OpenAI.'},
  {'entity_name': 'Instructions',
   'entity_type': 'Things',
   'entity_description': 'Complex instructions generated by Evol-Instruct, which are better than those created by humans.'}],
 'relationships': [{'source_entity': 'WizardLM',
   'target_entity': 'Evol-Instruct',
   'relationship_description': 'WizardLM is trained on a dataset of instructions generated by Evo

In [38]:
results['relationships']

recounstract_prompt = """Give following entities and relashionships:
{entities}

What entities and relationships are missing to infere all the knowledge in the following text:
{orginal_document}

Output format in JSON:
'''JSON
need_more_entities: Yes or No
new_entities_list: a list of NEW entities
new_relationships: in case of Yes
""".format(entities= results['relationships'],orginal_document = documents[0])

print(recounstract_prompt)

Give following entities and relashionships:
[{'source_entity': 'WizardLM', 'target_entity': 'Evol-Instruct', 'relationship_description': 'WizardLM is trained on a dataset of instructions generated by Evol-Instruct.', 'relationship_strength': 4}, {'source_entity': 'Evol-Instruct', 'target_entity': 'Instructions', 'relationship_description': 'Evol-Instruct creates complex instructions by gradually rewriting simpler ones.', 'relationship_strength': 5}, {'source_entity': 'WizardLM', 'target_entity': 'ChatGPT', 'relationship_description': "WizardLM outperforms OpenAI's ChatGPT in some tasks.", 'relationship_strength': 3}, {'source_entity': 'Instructions', 'target_entity': 'Humans', 'relationship_description': 'Instructions generated by Evol-Instruct are better than those created by humans.', 'relationship_strength': 4}]

What entities and relationships are missing to infere all the knowledge in the following text:
This text describes a new large language model (LLM) called WizardLM. WizardL

In [39]:
completion(recounstract_prompt)

{'need_more_entities': 'Yes',
 'new_entities_list': ['Large Language Model',
  'AI System',
  'Researchers',
  'Code',
  'Data',
  'OpenAI',
  'ChatGPT',
  'Tasks'],
 'new_relationships': [{'source_entity': 'WizardLM',
   'target_entity': 'Large Language Model',
   'relationship_description': 'WizardLM is a large language model.',
   'relationship_strength': 5},
  {'source_entity': 'Evol-Instruct',
   'target_entity': 'AI System',
   'relationship_description': 'Evol-Instruct is an AI system.',
   'relationship_strength': 5},
  {'source_entity': 'Researchers',
   'target_entity': 'Evol-Instruct',
   'relationship_description': 'Researchers developed Evol-Instruct.',
   'relationship_strength': 4},
  {'source_entity': 'Researchers',
   'target_entity': 'WizardLM',
   'relationship_description': 'Researchers developed WizardLM.',
   'relationship_strength': 4},
  {'source_entity': 'WizardLM',
   'target_entity': 'Code',
   'relationship_description': 'Code for WizardLM is publicly availa